In [9]:
k = input('>>> ')
print(k)

>>> two
two


In [1]:
def resolve_arg(program, argmode, base_offset, i, j):    
    if argmode == 0:
        return program[i+j]
    elif argmode == 1:
        return i+j
    elif argmode == 2:
        return base_offset + program[i+j]

def intcode(program, inlist=[], i=0):
    global next_move
    global robopose
    global pre_robopose
    global mazemap
    global openlist

    outbuf = []
    base_offset = 0
    
    while True:
        if i >= len(program):
            print('end of program')
            return program
        opcode = program[i]
        
        op = opcode % 100
        argmodes = []
        argcode = opcode // 100
        while argcode > 0:
            argmodes.append(argcode % 10)
            argcode = argcode // 10
            
        # ADD A B -> C
        # --------------------------------------------------------
        if op == 1:
            if len(argmodes) < 1:
                idx1 = resolve_arg(program, 0, base_offset, i, 1)
            else:
                idx1 = resolve_arg(program, argmodes[0], base_offset, i, 1)
            
            if len(argmodes) < 2:
                idx2 = resolve_arg(program, 0, base_offset, i, 2)
            else:
                idx2 = resolve_arg(program, argmodes[1], base_offset, i, 2)
            
            if len(argmodes) < 3:
                idx3 = resolve_arg(program, 0, base_offset, i, 3)
            else:
                idx3 = resolve_arg(program, argmodes[2], base_offset, i, 3)

            bigdex = max([idx1, idx2, idx3])
            if len(program) <= bigdex:
                program += [0]*(bigdex - len(program) + 1)
                
            arg1 = program[idx1]
            arg2 = program[idx2]                
            
            program[idx3] = arg1 + arg2
            i += 4
        # MULTIPLY A B ->
        # --------------------------------------------------------
        elif op == 2:
            if len(argmodes) < 1:
                idx1 = resolve_arg(program, 0, base_offset, i, 1)
            else:
                idx1 = resolve_arg(program, argmodes[0], base_offset, i, 1)
            
            
            if len(argmodes) < 2:
                idx2 = resolve_arg(program, 0, base_offset, i, 2)
            else:
                idx2 = resolve_arg(program, argmodes[1], base_offset, i, 2)

            if len(argmodes) < 3:
                idx3 = resolve_arg(program, 0, base_offset, i, 3)
            else:
                idx3 = resolve_arg(program, argmodes[2], base_offset, i, 3)
                
            bigdex = max([idx1, idx2, idx3])
            if len(program) <= bigdex:
                program += [0]*(bigdex - len(program) + 1)
                
            arg1 = program[idx1]
            arg2 = program[idx2]
            
            program[idx3] = arg1 * arg2
            i += 4
        # INPUT -> A
        # --------------------------------------------------------
        elif op == 3:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            
            if len(program) <= idx:
                program += [0]*(idx - len(program) + 1)

            if len(inlist) > 0:
                program[idx] = inlist[0]
                inlist = inlist[1:]
            else:
                program[idx] = int(input('>>> '))
                
            i += 2
            
        # OUTPUT A
        # --------------------------------------------------------
        elif op == 4:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            outbuf.append(program[idx])
            
            if program[idx] == 10:
                print(''.join([chr(e) for e in outbuf[:-1]]))
                outbuf = []
            i += 2
            
        # JUMP TRUE A != 0 -> B
        # --------------------------------------------------------
        elif op == 5:                
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx1 = resolve_arg(program, argmode, base_offset, i, 1)
            
            argmode = 0 if len(argmodes) < 2 else argmodes[1]
            idx2 = resolve_arg(program, argmode, base_offset, i, 2)
            
            bigdex = max([idx1, idx2])
            if len(program) <= bigdex:
                program += [0]*(bigdex - len(program) + 1)
            
            arg1 = program[idx1]
            arg2 = program[idx2]    
            if arg1 != 0:
                i = arg2
            else:
                i += 3
                
        # JUMP FALSE A == 0 -> B
        # --------------------------------------------------------
        elif op == 6:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            arg1 = program[idx]
            
            argmode = 0 if len(argmodes) < 2 else argmodes[1]
            idx = resolve_arg(program, argmode, base_offset, i, 2)
            arg2 = program[idx]
            
            if len(program) <= arg2:
                program += [0]*(arg2-len(program)+1)
                
            if arg1 == 0:
                i = arg2
            else:
                i += 3
                
        # LESS THAN A < B 1 -> C
        # --------------------------------------------------------
        elif op == 7:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            arg1 = program[idx]
            
            argmode = 0 if len(argmodes) < 2 else argmodes[1]
            idx = resolve_arg(program, argmode, base_offset, i, 2)
            arg2 = program[idx]
            
            argmode = 0 if len(argmodes) < 3 else argmodes[2]
            idx = resolve_arg(program, argmode, base_offset, i, 3)
                
            if len(program) <= idx:
                program += [0]*(idx-len(program)+1)
                
            if arg1 < arg2:
                program[idx] = 1
            else:
                program[idx] = 0
            i += 4
            
        # EQUALS A == B 1 -> C
        # --------------------------------------------------------
        elif op == 8:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            arg1 = program[idx]
            
            argmode = 0 if len(argmodes) < 2 else argmodes[1]
            idx = resolve_arg(program, argmode, base_offset, i, 2)
            arg2 = program[idx]
            
            argmode = 0 if len(argmodes) < 3 else argmodes[2]
            idx = resolve_arg(program, argmode, base_offset, i, 3)
                
            if len(program) <= idx:
                program += [0]*(idx-len(program)+1)
                
            if arg1 == arg2:
                program[idx] = 1
            else:
                program[idx] = 0
            i += 4
            
        # SHIFT RELATIVE BASE
        # --------------------------------------------------------
        elif op == 9:
            argmode = 0 if len(argmodes) < 1 else argmodes[0]
            idx = resolve_arg(program, argmode, base_offset, i, 1)
            arg1 = program[idx]
            base_offset += arg1
            if len(program) <= base_offset:
                program += [0]*(base_offset-len(program)+1)
            i += 2
            
        # HALT
        # --------------------------------------------------------
        elif op == 99:
            return outbuf
        
        # INVALID
        # --------------------------------------------------------
        else:
            print(program[i], 'is not an instruction')
            return outbuf

In [2]:
with open('calca_17_input.txt', 'r') as infile:
    instr = [row.strip() for row in infile][0]
    
program = [int(col) for col in instr.split(',')]
outbuf = intcode(program)

view = ''
vu = []
parvu = []
for e in outbuf:
    view += chr(e)
    if not (e == ord('\n')):
        parvu.append(e)
    else:
        if len(parvu) > 0:
            vu.append(parvu.copy())
        parvu = []
    
# print(view)

dirs = [[1, 0], [-1, 0], [0, 1], [0, -1]]

paramsum = 0

# print(len(vu), len(vu[0]))
for i in range(len(vu)):
    for j in range(len(vu[0])):
        if not (vu[i][j] == ord('#')):
            continue
        inter = True
        for di, dj in dirs:
            ii = i + di
            jj = j + dj
            if ii < 0 or jj < 0 or ii >= len(vu) or jj >= len(vu[0]):
                inter = False
                break
            else:
                inter = inter and (vu[ii][jj] == ord('#'))
        if inter:
            vu[i][j] = ord('O')
#             print('intersection in {}, {}'.format(i, j))
            paramsum += j*i

print(paramsum)
# print('   ', ''.join(['{:2d}'.format(i) for i in range(len(vu[0]))]))
for i, row in enumerate(vu):
    print('{:3d}'.format(i), ''.join([chr(col) for col in row]))

..................#####..........................
..................#...#..........................
..........###########.#..........................
..........#.......#.#.#..........................
..........#.......#########......................
..........#.........#.#...#......................
..........#############...#......................
....................#.....#......................
....................#.....#......................
....................#.....#......................
....................#.....#...............#####..
....................#.....#...............#...#..
....................#.....#...............#...#..
....................#.....#...............#...#..
..............#######.....#########.......#...#..
..............#...................#.......#...#..
..............#...................#.......#...#..
..............#...................#.......#...#..
..............#...................#.......#.#####
..............#...................#.......#.#.#.#


In [58]:
with open('calca_17_input.txt', 'r') as infile:
    instr = [row.strip() for row in infile][0]
    
sequence = 'A,B,A,C,A,B,C,A,B,C'
seq_a = 'R,12,R,4,R,10,R,12'
seq_b = 'R,6,L,8,R,10'
seq_c = 'L,8,R,4,R,4,R,6'

inbuf = []
for x in sequence:
    inbuf.append(ord(x))
inbuf.append(10)

for x in seq_a:
    inbuf.append(ord(x))
inbuf.append(10)

for x in seq_b:
    inbuf.append(ord(x))
inbuf.append(10)

for x in seq_c:
    inbuf.append(ord(x))
inbuf.append(10)

inbuf.append(ord('n'))
inbuf.append(10)

print(inbuf)
program = [int(col) for col in instr.split(',')]
program[0] = 2
outbuf = intcode(program, inlist=inbuf)
print(outbuf)

[65, 44, 66, 44, 65, 44, 67, 44, 65, 44, 66, 44, 67, 44, 65, 44, 66, 44, 67, 10, 82, 44, 49, 50, 44, 82, 44, 52, 44, 82, 44, 49, 48, 44, 82, 44, 49, 50, 10, 82, 44, 54, 44, 76, 44, 56, 44, 82, 44, 49, 48, 10, 76, 44, 56, 44, 82, 44, 52, 44, 82, 44, 52, 44, 82, 44, 54, 10, 110, 10]
..................#####..........................
..................#...#..........................
..........###########.#..........................
..........#.......#.#.#..........................
..........#.......#########......................
..........#.........#.#...#......................
..........#############...#......................
....................#.....#......................
....................#.....#......................
....................#.....#......................
....................#.....#...............#####..
....................#.....#...............#...#..
....................#.....#...............#...#..
....................#.....#...............#...#..
..............####

In [42]:
view = ''
vu = []
parvu = []
for e in outbuf:
    view += chr(e)
    if not (e == ord('\n')):
        parvu.append(e)
    else:
        if len(parvu) > 0:
            vu.append(parvu.copy())
        parvu = []
    
print(view)

..................#####..........................
..................#...#..........................
..........###########.#..........................
..........#.......#.#.#..........................
..........#.......#########......................
..........#.........#.#...#......................
..........#############...#......................
....................#.....#......................
....................#.....#......................
....................#.....#......................
....................#.....#...............#####..
....................#.....#...............#...#..
....................#.....#...............#...#..
....................#.....#...............#...#..
..............#######.....#########.......#...#..
..............#...................#.......#...#..
..............#...................#.......#...#..
..............#...................#.......#...#..
..............#...................#.......#.#####
..............#...................#.......#.#.#.#


In [37]:
view = ''
vu = []
parvu = []
for e in outbuf:
    view += chr(e)
    if not (e == ord('\n')):
        parvu.append(e)
    else:
        if len(parvu) > 0:
            vu.append(parvu.copy())
        parvu = []
    
# print(view)

dirs = [[1, 0], [-1, 0], [0, 1], [0, -1]]

paramsum = 0

# print(len(vu), len(vu[0]))
for i in range(len(vu)):
    for j in range(len(vu[0])):
        if not (vu[i][j] == ord('#')):
            continue
        inter = True
        for di, dj in dirs:
            ii = i + di
            jj = j + dj
            if ii < 0 or jj < 0 or ii >= len(vu) or jj >= len(vu[0]):
                inter = False
                break
            else:
                inter = inter and (vu[ii][jj] == ord('#'))
        if inter:
            vu[i][j] = ord('O')
#             print('intersection in {}, {}'.format(i, j))
            paramsum += j*i

print(paramsum)
# print('   ', ''.join(['{:2d}'.format(i) for i in range(len(vu[0]))]))
for i, row in enumerate(vu):
    print('{:3d}'.format(i), ''.join([chr(col) for col in row]))

>>> 65
>>> 10
>>> 82
>>> 44
>>> 49
>>> 48
>>> 10
>>> 110


IndexError: list index out of range